#Intro
Penney's game is a 2 player nontransitive game that can be done using a deck of cards or filling some coins. It is needed that each binary state is roughly 50-50

Two players start by guessing a string of binary states such as HHT or TTT  
and the goal is to see who's string appears first. Each string is known to everyone


When first published, Penney's game uses a string of 3 heads or tails. It has also been shown that red-black cards of a deck of cards is approximate to the probabilities

#Goal
This python notebooks seeks 
1. To generalize the game to a string of length n by utilizing markov chains. 
2. To generalize the game to include m players

In [484]:
import numpy as np
import pandas as pd

A markov chain can be represented as an adjacency matrix.  
I will be using binary to represent the states where H=0 and T=1  


So 010 = HTH  


The matrix forms a diagonal matrix that resembles some stairsteps

In [485]:
#generating markov matrix

#length of subset of interest
length = 3

#side length of matrix
mtx_length = 2**length


def make_markov(mtx_length):

  #get identity/2
  step1 = np.eye(mtx_length//2, dtype=float)/2

  #interweave columns of identity/2 with itself
  step2 = np.empty((step1.shape[0], step1.shape[0]*2))
  step2[:,1::2] = step1 #interweaving
  step2[:,::2] = step1 

  #append a copy to the bottom
  step3 = np.append(step2,step2, axis=0)

  #done
  return step3


def get_states(mtx_length):
  states = {}
  for i in range(mtx_length):
    states[bin(i)] = i
  return states


matrix = make_markov(mtx_length)
states = get_states(mtx_length)
bin_states = list(states.keys())


#displaying markov matrix in a dataframe
matrix_df = pd.DataFrame(matrix)
matrix_df.set_axis(bin_states, axis=0, inplace=True)
matrix_df.set_axis(bin_states, axis=1, inplace=True)

matrix_df

,0b0,0b1,0b10,0b11,0b100,0b101,0b110,0b111
0b0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0
0b1,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0
0b10,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0
0b11,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5
0b100,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0
0b101,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0
0b110,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0
0b111,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5


It is to be interpreted that moving along the columns is player 1 and moving across the rows is player 2

In [486]:

#function to get the probabilities when comparing 2 strings
def get_probabilities(players, mtx_length=mtx_length):
  '''
  me, you are binary strings like 0b1 or 0b111
  '''

  states = get_states(mtx_length)
  bin_states = list(states.keys())

  #modifying matrix with ending conditions, the 2 states considered

  I = np.eye(mtx_length,mtx_length, dtype = float)

  mtx_n = make_markov(mtx_length)

  #turning binary into decimal
  player_states = [states[i] for i in players]

  for j in player_states:
    mtx_n[j,:] = I[j,:]


  #print("Markov Matrix\n",mtx_n)

  #each state is equally likely
  start = np.ones((1,mtx_length))/mtx_length

  #we want the markov matrix to converge with repeat multiplications
  #100 is close enough when we round the answer
  n = 10
  M = mtx_n
  for _ in range(n):
    M = np.dot(M,M)


  #P = markov matrix
  #winnings = [0.125,...,0.125]*P^n, as n -> infinity
  winnings = np.dot(start,M.round(5)).round(5)

  return [winnings[:,i][0] for i in player_states]


get_probabilities(['0b0', '0b1','0b100','0b111'])

[0.125, 0.125, 0.45, 0.3]

Here, the output of get_probabilities comes out as (P(player 1 wins), P(player 2 wins))

#Getting the probabilities for each state

the code below works out every type of pairing and stores them all into a dataframe

In [487]:


all_probablities = []
states = get_states(mtx_length)
bin_states = list(states.keys())


for row in range(mtx_length):
  temp = []
  for col in range(mtx_length):
    temp.append(get_probabilities([bin_states[col], bin_states[row]]))
  all_probablities.append(temp)


if len(all_probablities.shape) == 2:
  winning_probabilities = pd.DataFrame(all_probablities).T
  winning_probabilities.set_axis(bin_states, axis=0, inplace=True) #renaming row/col
  winning_probabilities.set_axis(bin_states, axis=1, inplace=True)

winning_probabilities

,0b0,0b1,0b10,0b11,0b100,0b101,0b110,0b111
0b0,"[1.0, 1.0]","[0.5, 0.5]","[0.4, 0.6]","[0.4, 0.6]","[0.125, 0.875]","[0.41667, 0.58333]","[0.3, 0.7]","[0.5, 0.5]"
0b1,"[0.5, 0.5]","[1.0, 1.0]","[0.66667, 0.33333]","[0.66667, 0.33333]","[0.25, 0.75]","[0.625, 0.375]","[0.5, 0.5]","[0.7, 0.3]"
0b10,"[0.6, 0.4]","[0.33333, 0.66667]","[1.0, 1.0]","[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]","[0.375, 0.625]","[0.58333, 0.41667]"
0b11,"[0.6, 0.4]","[0.33333, 0.66667]","[0.5, 0.5]","[1.0, 1.0]","[0.5, 0.5]","[0.5, 0.5]","[0.75, 0.25]","[0.875, 0.125]"
0b100,"[0.875, 0.125]","[0.75, 0.25]","[0.5, 0.5]","[0.5, 0.5]","[1.0, 1.0]","[0.5, 0.5]","[0.33333, 0.66667]","[0.6, 0.4]"
0b101,"[0.58333, 0.41667]","[0.375, 0.625]","[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]","[1.0, 1.0]","[0.33333, 0.66667]","[0.6, 0.4]"
0b110,"[0.7, 0.3]","[0.5, 0.5]","[0.625, 0.375]","[0.25, 0.75]","[0.66667, 0.33333]","[0.66667, 0.33333]","[1.0, 1.0]","[0.5, 0.5]"
0b111,"[0.5, 0.5]","[0.3, 0.7]","[0.41667, 0.58333]","[0.125, 0.875]","[0.4, 0.6]","[0.4, 0.6]","[0.5, 0.5]","[1.0, 1.0]"


In [508]:
def for_loop(mtx_length, n_loops):
    if n < 0:
        for x in range(mtx_length):
          print(x,n_loops)
          for_loop(mtx_length, n_loops - 1)
    else:
       print('done')

for_loop(4, 4)


done


#Finding the winning strategy

The winning strategy for player 1 can be found by looking at every option player 2 has and finding the string that beats it.



In [488]:
i_win = []

for col in range(mtx_length): #each col is a string player 2 could play
  temp = None
  for row in range(mtx_length): #each row is a string player 1 could play
    prob = winning_probabilities.iat[row, col]
  
    if temp is None and prob[0] != 1:
      temp = [prob, row, col]
    elif prob[0] == 1:
      pass
    elif temp[0][0] < prob[0]:
      temp = [prob, row, col]

  i_win.append([f'{bin_states[temp[1]]} beats {bin_states[temp[2]]}',temp[0]])

#i_win_df = pd.DataFrame(i_win).T

i_win


[['0b100 beats 0b0', [0.875, 0.125]],
 ['0b100 beats 0b1', [0.75, 0.25]],
 ['0b1 beats 0b10', [0.66667, 0.33333]],
 ['0b1 beats 0b11', [0.66667, 0.33333]],
 ['0b110 beats 0b100', [0.66667, 0.33333]],
 ['0b110 beats 0b101', [0.66667, 0.33333]],
 ['0b11 beats 0b110', [0.75, 0.25]],
 ['0b11 beats 0b111', [0.875, 0.125]]]

If there is a string that is greater than 50%, then player 1 will win most of the time if going second in response to observing player 2's string

#Verifying the math
The probabilities above are made theoretically, so it would be nice to see it it can be observed

In [489]:

num_flips = 20
num_trials = 10000

walk = np.random.randint(0,2,(num_trials,num_flips))


def list2str(array):
  decimals = []

  if len(array.shape) == 1:
    num_flips = array.shape[0]
    num_trials = 1
  else:
    num_flips = array.shape[1]
    num_trials = array.shape[0]

  for trial in range(num_trials):
    walk = []
    for flip in range(3,num_flips+1):
      subwalk = int('0b'+''.join(map(str,array[trial,(flip-3):flip])),2)
      #subwalk = '0b'+''.join(map(str,array[trial,(flip-3):flip]))
      walk.append(subwalk)
    decimals.append(walk)

  return np.array(decimals)

def check_str(players, array, string_length=3):
  '''
  Input:
  players=list of binary strings
  array=decimal translating of random walk per subwalk
  string_length=length of string used for the game
  '''
  states = get_states(2**string_length) #binary to decimal
  bin_states = list(states.keys()) #decimal to binary

  #make a dictionary from list of strings
  count = dict.fromkeys(players,0)

  for row in range(array.shape[0]):
    for col in range(array.shape[1]):
      sub_state = bin_states[array[row,col]] #decimal to binary
      if sub_state in players:
        count[sub_state] += 1
        break
      else:
        assert 'Unknown Error'

  total = np.sum(list(count.values()))

  for i in players:
    count[i] = count[i]/total
  return count


#int('0b'+''.join(map(str, walk)),2)

print(check_str(['0b0','0b1','0b10'], array = list2str(walk)))



{'0b0': 0.3407288424856942, '0b1': 0.32436502359200886, '0b10': 0.33490613392229696}


Testing out a few comparison seems to agree with theory, nice!

In [490]:
get_states(10).values()

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])